#Wave Height Predictions with ARIMA

## Introduction


The goal is to use machine learning to build a time-series model that can be used to predict wave height measurements for the selected buoy area. We'll be building an ARIMA model.

The dataset comes from the [National Data Buoy Center](https://www.ndbc.noaa.gov/).

The buoy used for this example is Buoy #[51101](https://www.ndbc.noaa.gov/station_page.php?station=51101), which is located Northwest of Kauai, Hawaii.

Testing.

### ARIMA

You can read this article for more info.
https://medium.com/analytics-vidhya/arima-for-dummies-ba761d59a051


# Imports and Data

## Imports

The below code chunk loads in all your packages and imports, make sure to run this every time you start your session.

Packages include all the tools you need to successfully run all your code and analysis.

In [38]:
#pip install statsmodels
#pip install pmdarima
#pip intall seaborn
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn import linear_model

from pandas import to_datetime
import datetime
from datetime import datetime
import pandas as pd
#import pmdarima as pm
import itertools
import warnings
import matplotlib.pyplot as plt
from matplotlib import pyplot

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_predict

warnings.filterwarnings('ignore')
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


## Data

To load in our data, we use the pandas function read_csv to get the data directly from the National Data Buoy Center and are assigning it to the variable called *df1*.

A variable is a place/object where we can store information.

In [39]:
df1 = pd.read_csv("https://www.ndbc.noaa.gov/view_text_file.php?filename=41049h2017.txt.gz&dir=data/historical/stdmet/"
                  , delimiter= '\s+', index_col=False) # reading in the data

In [40]:
df_23=pd.read_csv("https://www.ndbc.noaa.gov/view_text_file.php?filename=41049h2023.txt.gz&dir=data/historical/stdmet/"
                  , delimiter= '\s+', index_col=False) # reading in the data
df_22=pd.read_csv("https://www.ndbc.noaa.gov/view_text_file.php?filename=41049h2022.txt.gz&dir=data/historical/stdmet/"
                  , delimiter= '\s+', index_col=False) # reading in the data
df_21=pd.read_csv("https://www.ndbc.noaa.gov/view_text_file.php?filename=41049h2021.txt.gz&dir=data/historical/stdmet/"
                  , delimiter= '\s+', index_col=False) # reading in the data
df_20=pd.read_csv("https://www.ndbc.noaa.gov/view_text_file.php?filename=41049h2020.txt.gz&dir=data/historical/stdmet/"
                  , delimiter= '\s+', index_col=False) # reading in the data

In [41]:
df_23.head() # looking at first 5 rows

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,#yr,mo,dy,hr,mn,degT,m/s,m/s,m,sec,sec,degT,hPa,degC,degC,degC,mi,ft
1,2023,01,01,00,00,76,7.0,8.9,99.00,99.00,99.00,999,1023.3,23.1,24.4,18.6,99.0,99.00
2,2023,01,01,00,10,82,6.8,9.7,1.93,9.09,5.88,111,1023.4,23.1,24.4,18.9,99.0,99.00
3,2023,01,01,00,20,75,6.6,8.2,99.00,99.00,99.00,999,1023.4,23.1,24.5,18.9,99.0,99.00
4,2023,01,01,00,30,77,7.0,8.6,99.00,99.00,99.00,999,1023.5,23.1,24.5,18.9,99.0,99.00


In [42]:
df_22

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,#yr,mo,dy,hr,mn,degT,m/s,m/s,m,sec,sec,degT,hPa,degC,degC,degC,mi,ft
1,2022,01,01,00,00,285,5.8,7.1,99.00,99.00,99.00,999,1017.9,24.0,24.3,20.8,99.0,99.00
2,2022,01,01,00,10,286,6.1,7.4,99.00,99.00,99.00,999,1018.1,24.0,24.3,20.7,99.0,99.00
3,2022,01,01,00,20,289,6.7,8.1,99.00,99.00,99.00,999,1018.4,24.0,24.3,20.5,99.0,99.00
4,2022,01,01,00,30,287,6.8,8.7,99.00,99.00,99.00,999,1018.2,24.0,24.3,20.6,99.0,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52528,2022,12,31,23,10,78,7,9,1.94,7.69,6.01,86,1022.9,23,24.4,18.6,99,99
52529,2022,12,31,23,20,78,6.7,9,99,99,99,999,1023,23,24.4,18.5,99,99
52530,2022,12,31,23,30,75,6.8,8.7,99,99,99,999,1023,23,24.5,18.5,99,99
52531,2022,12,31,23,40,79,6.6,8.5,1.92,10,6.1,109,1023.1,23.1,24.4,18.5,99,99


In [43]:
df_21

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,#yr,mo,dy,hr,mn,degT,m/s,m/s,m,sec,sec,degT,hPa,degC,degC,degC,mi,ft
1,2021,01,01,00,00,54,8.5,10.8,99.00,99.00,99.00,999,9999.0,999.0,24.0,999.0,99.0,99.00
2,2021,01,01,00,10,56,8.9,11.6,99.00,99.00,99.00,999,9999.0,999.0,24.0,999.0,99.0,99.00
3,2021,01,01,00,20,61,9.5,12.1,99.00,99.00,99.00,999,9999.0,999.0,24.0,999.0,99.0,99.00
4,2021,01,01,00,30,61,9.2,12.0,99.00,99.00,99.00,999,9999.0,999.0,24.0,999.0,99.0,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51505,2021,12,31,23,10,291,7.1,8.5,99,99,99,999,1018,24,24.4,20.5,99,99
51506,2021,12,31,23,20,291,6.1,7.4,99,99,99,999,1018,24,24.4,20.7,99,99
51507,2021,12,31,23,30,292,5.9,7.4,99,99,99,999,1018,24,24.4,20.6,99,99
51508,2021,12,31,23,40,287,6,7.4,1.6,12.12,6.52,38,1018.1,24,24.4,20.7,99,99


In [44]:
df_20

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,#yr,mo,dy,hr,mn,degT,m/s,m/s,m,sec,sec,degT,hPa,degC,degC,degC,mi,ft
1,2020,01,01,00,00,180,3.4,4.8,99.00,99.00,99.00,999,1019.0,999.0,999.0,999.0,99.0,99.00
2,2020,01,01,00,10,179,3.5,5.0,99.00,99.00,99.00,999,1019.0,999.0,999.0,999.0,99.0,99.00
3,2020,01,01,00,20,172,3.6,5.0,99.00,99.00,99.00,999,1019.0,999.0,999.0,999.0,99.0,99.00
4,2020,01,01,00,30,173,3.5,4.9,99.00,99.00,99.00,999,1019.1,999.0,999.0,999.0,99.0,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52166,2020,12,31,23,10,50,9.2,11.2,99,99,99,999,9999,999,24,999,99,99
52167,2020,12,31,23,20,53,8.7,11.5,99,99,99,999,9999,999,24,999,99,99
52168,2020,12,31,23,30,56,8.5,11,99,99,99,999,9999,999,24,999,99,99
52169,2020,12,31,23,40,53,7.9,10.7,2.89,10.81,7.21,15,9999,999,24,999,99,99


In [45]:
print(len(df_23))
print(len(df_22))
print(len(df_21))
print(len(df_20))

52441
52533
51510
52171


# Exploring the Data

For this project, the target variable we'll focus on is WVHT (wave height in meters).

In [46]:
df_23.info() # looking at basic information about your variable

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   #YY     52441 non-null  object
 1   MM      52441 non-null  object
 2   DD      52441 non-null  object
 3   hh      52441 non-null  object
 4   mm      52441 non-null  object
 5   WDIR    52441 non-null  object
 6   WSPD    52441 non-null  object
 7   GST     52441 non-null  object
 8   WVHT    52441 non-null  object
 9   DPD     52441 non-null  object
 10  APD     52441 non-null  object
 11  MWD     52441 non-null  object
 12  PRES    52441 non-null  object
 13  ATMP    52441 non-null  object
 14  WTMP    52441 non-null  object
 15  DEWP    52441 non-null  object
 16  VIS     52441 non-null  object
 17  TIDE    52441 non-null  object
dtypes: object(18)
memory usage: 7.2+ MB


From the above information, we learn that the dataset currently has 8,645 rows and 18 columns.

In [47]:
df_23.head() # looking at the first five rows of your data

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,#yr,mo,dy,hr,mn,degT,m/s,m/s,m,sec,sec,degT,hPa,degC,degC,degC,mi,ft
1,2023,01,01,00,00,76,7.0,8.9,99.00,99.00,99.00,999,1023.3,23.1,24.4,18.6,99.0,99.00
2,2023,01,01,00,10,82,6.8,9.7,1.93,9.09,5.88,111,1023.4,23.1,24.4,18.9,99.0,99.00
3,2023,01,01,00,20,75,6.6,8.2,99.00,99.00,99.00,999,1023.4,23.1,24.5,18.9,99.0,99.00
4,2023,01,01,00,30,77,7.0,8.6,99.00,99.00,99.00,999,1023.5,23.1,24.5,18.9,99.0,99.00


In [48]:
df_22

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,#yr,mo,dy,hr,mn,degT,m/s,m/s,m,sec,sec,degT,hPa,degC,degC,degC,mi,ft
1,2022,01,01,00,00,285,5.8,7.1,99.00,99.00,99.00,999,1017.9,24.0,24.3,20.8,99.0,99.00
2,2022,01,01,00,10,286,6.1,7.4,99.00,99.00,99.00,999,1018.1,24.0,24.3,20.7,99.0,99.00
3,2022,01,01,00,20,289,6.7,8.1,99.00,99.00,99.00,999,1018.4,24.0,24.3,20.5,99.0,99.00
4,2022,01,01,00,30,287,6.8,8.7,99.00,99.00,99.00,999,1018.2,24.0,24.3,20.6,99.0,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52528,2022,12,31,23,10,78,7,9,1.94,7.69,6.01,86,1022.9,23,24.4,18.6,99,99
52529,2022,12,31,23,20,78,6.7,9,99,99,99,999,1023,23,24.4,18.5,99,99
52530,2022,12,31,23,30,75,6.8,8.7,99,99,99,999,1023,23,24.5,18.5,99,99
52531,2022,12,31,23,40,79,6.6,8.5,1.92,10,6.1,109,1023.1,23.1,24.4,18.5,99,99


In [49]:
df_21

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,#yr,mo,dy,hr,mn,degT,m/s,m/s,m,sec,sec,degT,hPa,degC,degC,degC,mi,ft
1,2021,01,01,00,00,54,8.5,10.8,99.00,99.00,99.00,999,9999.0,999.0,24.0,999.0,99.0,99.00
2,2021,01,01,00,10,56,8.9,11.6,99.00,99.00,99.00,999,9999.0,999.0,24.0,999.0,99.0,99.00
3,2021,01,01,00,20,61,9.5,12.1,99.00,99.00,99.00,999,9999.0,999.0,24.0,999.0,99.0,99.00
4,2021,01,01,00,30,61,9.2,12.0,99.00,99.00,99.00,999,9999.0,999.0,24.0,999.0,99.0,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51505,2021,12,31,23,10,291,7.1,8.5,99,99,99,999,1018,24,24.4,20.5,99,99
51506,2021,12,31,23,20,291,6.1,7.4,99,99,99,999,1018,24,24.4,20.7,99,99
51507,2021,12,31,23,30,292,5.9,7.4,99,99,99,999,1018,24,24.4,20.6,99,99
51508,2021,12,31,23,40,287,6,7.4,1.6,12.12,6.52,38,1018.1,24,24.4,20.7,99,99


In [50]:
df_20

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,#yr,mo,dy,hr,mn,degT,m/s,m/s,m,sec,sec,degT,hPa,degC,degC,degC,mi,ft
1,2020,01,01,00,00,180,3.4,4.8,99.00,99.00,99.00,999,1019.0,999.0,999.0,999.0,99.0,99.00
2,2020,01,01,00,10,179,3.5,5.0,99.00,99.00,99.00,999,1019.0,999.0,999.0,999.0,99.0,99.00
3,2020,01,01,00,20,172,3.6,5.0,99.00,99.00,99.00,999,1019.0,999.0,999.0,999.0,99.0,99.00
4,2020,01,01,00,30,173,3.5,4.9,99.00,99.00,99.00,999,1019.1,999.0,999.0,999.0,99.0,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52166,2020,12,31,23,10,50,9.2,11.2,99,99,99,999,9999,999,24,999,99,99
52167,2020,12,31,23,20,53,8.7,11.5,99,99,99,999,9999,999,24,999,99,99
52168,2020,12,31,23,30,56,8.5,11,99,99,99,999,9999,999,24,999,99,99
52169,2020,12,31,23,40,53,7.9,10.7,2.89,10.81,7.21,15,9999,999,24,999,99,99


Let's look to see if we have NA values (missing values) in our dataset.

We're making another variable called *na_values*, to see if any column has NA values.

In [51]:
na_values = pd.DataFrame(df_23.isna().sum()) # looking at na values
na_values 

,0
#YY,0
MM,0
DD,0
hh,0
mm,0
WDIR,0
WSPD,0
GST,0
WVHT,0
DPD,0


# Data Cleaning

We need to clean our data to help our analysis run smoother and format it properly.

We'll be deleting the first two rows of our dataset as it just repeats the column names and has a row from 2016, and assigning it to a new variable called df1_clean.

In [52]:
df1_clean = df_20.iloc[2:] # dropping the first two rows
df1_clean 

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
2,2020,01,01,00,10,179,3.5,5.0,99.00,99.00,99.00,999,1019.0,999.0,999.0,999.0,99.0,99.00
3,2020,01,01,00,20,172,3.6,5.0,99.00,99.00,99.00,999,1019.0,999.0,999.0,999.0,99.0,99.00
4,2020,01,01,00,30,173,3.5,4.9,99.00,99.00,99.00,999,1019.1,999.0,999.0,999.0,99.0,99.00
5,2020,01,01,00,40,178,3.5,4.5,1.31,10.81,7.25,25,1019.1,999.0,999.0,999.0,99.0,99.00
6,2020,01,01,00,50,169,4.1,5.7,99.00,99.00,99.00,999,1019.0,999.0,999.0,999.0,99.0,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52166,2020,12,31,23,10,50,9.2,11.2,99,99,99,999,9999,999,24,999,99,99
52167,2020,12,31,23,20,53,8.7,11.5,99,99,99,999,9999,999,24,999,99,99
52168,2020,12,31,23,30,56,8.5,11,99,99,99,999,9999,999,24,999,99,99
52169,2020,12,31,23,40,53,7.9,10.7,2.89,10.81,7.21,15,9999,999,24,999,99,99


In [56]:
df1_clean= df_23

As you can see, there are separate columns for year, month, day, hour, and minute. We need to combine these columns in order to create a single datetime column.

In [57]:
df1_clean = df1_clean.astype({'#YY' : str, 'MM' : str, 'DD' : str, 'hh' : str, 'mm' : str}) # converting these columns to a string data type
df1_clean['Date'] = df1_clean['#YY'] + '-' + df1_clean['MM'] + '-' + df1_clean['DD'] + ' ' + df1_clean['hh'] + ':' + df1_clean['mm'] # combining all the datetime info into a Date column
df1_clean = df1_clean.drop(columns=['#YY','MM','DD','hh','mm']) # deleting the old date and time columns
df1_clean

,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE,Date
0,degT,m/s,m/s,m,sec,sec,degT,hPa,degC,degC,degC,mi,ft,#yr-mo-dy hr:mn
1,76,7.0,8.9,99.00,99.00,99.00,999,1023.3,23.1,24.4,18.6,99.0,99.00,2023-01-01 00:00
2,82,6.8,9.7,1.93,9.09,5.88,111,1023.4,23.1,24.4,18.9,99.0,99.00,2023-01-01 00:10
3,75,6.6,8.2,99.00,99.00,99.00,999,1023.4,23.1,24.5,18.9,99.0,99.00,2023-01-01 00:20
4,77,7.0,8.6,99.00,99.00,99.00,999,1023.5,23.1,24.5,18.9,99.0,99.00,2023-01-01 00:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52436,336,9,11.5,2.44,10,7.32,322,1020.5,21.5,24.9,19.6,99,99,2023-12-31 23:10
52437,337,8.5,10.5,99,99,99,999,1020.4,21.2,24.9,19.4,99,99,2023-12-31 23:20
52438,337,7.5,9.5,99,99,99,999,1020.4,20.9,25,19.4,99,99,2023-12-31 23:30
52439,343,6.9,8.5,2.57,10,7.61,309,1020.4,21,25,19.4,99,99,2023-12-31 23:40


Now we have one column with all the information we need. To move on, we need the data type for that column to be datetime format.

In [58]:
df1_clean['Date'] = pd.to_datetime(df1_clean['Date'])
df1_clean

ValueError: ('Unknown string format:', '#yr-mo-dy hr:mn')

In [59]:
df1_clean.info() # looking at updated info about our clean dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   WDIR    52441 non-null  object
 1   WSPD    52441 non-null  object
 2   GST     52441 non-null  object
 3   WVHT    52441 non-null  object
 4   DPD     52441 non-null  object
 5   APD     52441 non-null  object
 6   MWD     52441 non-null  object
 7   PRES    52441 non-null  object
 8   ATMP    52441 non-null  object
 9   WTMP    52441 non-null  object
 10  DEWP    52441 non-null  object
 11  VIS     52441 non-null  object
 12  TIDE    52441 non-null  object
 13  Date    52441 non-null  object
dtypes: object(14)
memory usage: 5.6+ MB


We see that the date column is a datetime data type, and the dataset is now 8,643 rows and 14 columns.

# ARIMA

## ARIMA Prep

In ARIMA, we're only going to be looking at two columns - the date and the WVHT columns (wave height is what we're trying to predict). We're going to select those two columns only and store it in a new variable called *arima_data*.

In [60]:
arima_data = df1_clean[['Date', 'WVHT']] # selecting columns
arima_data.head() # looking at first five rows of arima_data

,Date,WVHT
0,#yr-mo-dy hr:mn,m
1,2023-01-01 00:00,99.00
2,2023-01-01 00:10,1.93
3,2023-01-01 00:20,99.00
4,2023-01-01 00:30,99.00


Next, we want to set the date as the index (the row labels).

In [61]:
arima_data = arima_data.set_index('Date') # setting date column to index
arima_data

,WVHT
Date,
#yr-mo-dy hr:mn,m
2023-01-01 00:00,99.00
2023-01-01 00:10,1.93
2023-01-01 00:20,99.00
2023-01-01 00:30,99.00
...,...
2023-12-31 23:10,2.44
2023-12-31 23:20,99
2023-12-31 23:30,99


Let's try to see what our data looks like.

Our data represents wave heights in our selected buoy area for the year 2017.

In [62]:
arima_data['WVHT'] = arima_data['WVHT'].astype(float) # changing the datatype for WVHT for a nicer look on the plot
plt.figure(figsize=(10,6)) # setting up the dimentions for our plot
plt.title('2017 Wave Height Measurements (m) for South Bermuda') # setting up a title
plt.plot(arima_data['WVHT']) # plotting our data

ValueError: could not convert string to float: 'm'

## Splitting Data for Training and Testing

For supervised machine learning, it's common to split your data for training and testing.

We'll be splitting our data 80-20, meaning 80% of our data will be used to train our ARIMA model while we use the other 20% to test how well our model did.

The training data will be stored in a new variable called *arima_train* while the testing data will be stored in *arima_test*.

In [63]:
arima_tts = arima_data['WVHT']
arima_train = arima_tts[:int(len(arima_data)*0.8)] # selecting 80% of arima_data for training
arima_test = arima_tts[int(len(arima_data)*0.8):] # selecting the last 20% of data for testing

Let's double check to make sure our train-test-split went smoothly (that the numbers add up).

In [64]:
print(len(arima_train)) # printing the length of arima_train
print(len(arima_test)) # printing the length of arima_test
print(len(arima_data)) # arima_train + arima_test should equal arima_data

41952
10489
52441


## ARIMA Parameters and Testing for Stationarity

ARIMA models have 3 parameters (p, q, d):

- AR = p (found from pacf plot)
- MA = q (found from acf plot)
- The number of differencing = d

------

In order to determine d, we need to figure out if our data is stationary.

Stationary means that your data has a constant mean, variance, and covariance that is independent of time. We can test for stationarity using the Augmented Dickey Fuller (ADF) test.

If your data is **NOT** stationary, we would need to difference it.


In [65]:
adf_test = adfuller(arima_data['WVHT']) # performing adf test
print('ADF Statistic: ', adf_test[0]) # printing results
print('P-value: ', adf_test[1])

ValueError: could not convert string to float: 'm'

If your p-value is > .05, your time series is not stationary.

If it is < .05, your time series is stationary.

Based on the p-value, our data is *not* stationary.

Because it is not stationary, we need to perform differencing (in order to figure out what the d parameter will be).

In [66]:
arima_train_diff = arima_train.diff().dropna() # first order differencing

plt.figure(figsize=(10, 6)) # setting up plot size
plt.title('First Order Differencing')
plt.plot(arima_train_diff['WVHT']) # plotting differencing results

TypeError: unsupported operand type(s) for -: 'str' and 'str'

Since we performed differencing *once*, our d parameter will be equal to 1. Now let's figure out p and q using autocorrelation and partial correlation plots.

In [27]:
acf_plot = plot_acf(arima_train.diff().dropna()) # autocorrelation plot using training data

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [28]:
pacf_plot = plot_pacf(arima_train.diff().dropna()) # partial autocorrelation plot on training data

TypeError: unsupported operand type(s) for -: 'str' and 'str'

Based on this plot, we can see that 1 is significant, so p will equal 1.

Based on the acf_plot, it appears to tail off at 2, so we will use 2 for q.

Our parameters for now will be 1,2,1.

### Auto Arima

There's a cool function called auto_arima that is a statistical algorithm for determining the optimal parameters for ARIMA.

Running this code might take a minute.

In [29]:
import pmdarima as pm # importing the package

auto_arima = pm.auto_arima(arima_train, stepwise = False, seasonal = False) # using auto arima on the training data

auto_arima # printing variable

ImportError: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'OpenSSL 1.0.2k-fips  26 Jan 2017'. See: https://github.com/urllib3/urllib3/issues/2168

Based on these results, the optimal parameters are 1,1,2 (we were close!). We'll use that instead when building out our ARIMA model.

## Building the Model

Below we use this code to build and print out the predicted and expected wave heights for the test.

This code might take a couple of minutes.

In [30]:
# monkey patch around bug in ARIMA class
def __getnewargs__(self):
 return ((self.endog),(self.k_lags, self.k_diff, self.k_ma))
ARIMA.__getnewargs__ = __getnewargs__

In [31]:
from statsmodels.tsa.arima_model import ARIMAResults

In [32]:
waveheight_values = arima_data.WVHT
history = [x for x in arima_train]
predictions = list()
for t in range(len(arima_test)):
  model = ARIMA(history, order=(1,1,2)) # input the paramaters here within order=()
  model_fit = model.fit()
  output = model_fit.forecast()
  yhat = output[0]
  predictions.append(yhat)
  obs = arima_test[t]
  history.append(obs)
  print('predicted=%f, expected=%f' % (yhat, obs))

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [33]:
import csv
with open('predictions.csv', 'w', newline='') as file:
    # Step 4: Using csv.writer to write the list to the CSV file
    writer = csv.writer(file)
    writer.writerow(predictions) # Use writerow for single list

In [32]:
# save model
model_fit.save('model.pkl')
# load model
loaded = ARIMAResults.load('model.pkl')

AttributeError: 'ARIMA' object has no attribute 'k_lags'

## Visualization

After the model completes its forecast, create a plot to visualize the difference between the predicted values and expected vales for wave heights.

In [ ]:
test_x = arima_test.index # creating an index variable so that it's the dates

plt.figure (figsize=(10,6)) # setting up the plot size

plt.plot(test_x, arima_test, label='Expected') # plotting expected values
plt.plot(test_x, predictions, color='red', label='Prediction') # plotting predicted values
plt.xlabel('Date')
plt.ylabel('Wave Height')
plt.title('Difference Between Predicted and Actual Wave Heights')
plt.legend() # adding a legend to the plot

plt.xticks(rotation=45) # rotating the x labels for better visualization

plt.show()

In [ ]:
import plotly.graph_objects as go
# Assuming arima_train, arima_test, and predictions are Pandas Series with datetime index
train_x = arima_train.index
test_x = arima_test.index
fig = go.Figure()
# Add traces for training, expected, and predicted values
fig.add_trace(go.Scatter(x=train_x, y=arima_train, mode='lines', name='Training', line=dict(color='green')))
fig.add_trace(go.Scatter(x=test_x, y=arima_test, mode='lines', name='Expected'))
fig.add_trace(go.Scatter(x=test_x, y=predictions, mode='lines', name='Prediction', line=dict(color='red')))
# Update layout for better visualization
fig.update_layout(
    title='Difference Between Predicted and Actual Wave Heights',
    xaxis_title='Date',
    yaxis_title='Wave Height',
    legend_title='Legend',
    xaxis=dict(tickangle=45),
    autosize=False,
    width=1000,
    height=600,
)
# Show the plot
fig.show()

## Model Evaluation

In machine learning, there are different ways to evaluate how well your model performed depending on what type of model you use.

For this project, we'll use mean absolute error and root squared value to determine how well our model did.

You can review this [resource](https://www.analyticsvidhya.com/blog/2021/10/evaluation-metric-for-regression-models/) to understand the metrics better.

In [ ]:
mae = mean_absolute_error(arima_test, predictions) # calculating mean absolute error
r_sq = r2_score(arima_test, predictions) # calculating root squared value
print('Test MAE: ', mae)
print('Test R^2: ', r_sq)

Based on these results and the visualization, this model did pretty good!

## Forecasting Values for the Future

Now that we've fitted, trained, tested, and evaluated our model, let's try to forecast future values.